In [1]:
import numpy as np
import pandas as pd
import glob
import nltk 
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
import shutil
import random
from wordcloud import WordCloud
import gensim
import seaborn as sns
from textblob import TextBlob
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.sparse import hstack


nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_dff = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [3]:
train_df=train_dff[:]

In [4]:
# from collections import Counter
# from imblearn.under_sampling import RandomUnderSampler
# import pandas as pd
# desired_count = 20000 

# print("Before undersampling:", Counter(train_df['overall']))
# most_common_class, most_common_count = Counter(train_df['overall']).most_common(1)[0]
# sampling_strategy = {label: min(desired_count, count) for label, count in Counter(train_df['overall']).items()}
# rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
# undersampled_X, undersampled_y = rus.fit_resample(train_df.drop(columns=['overall']), train_df['overall'])
# train_df = pd.concat([pd.DataFrame(undersampled_X), pd.DataFrame({'overall': undersampled_y})], axis=1)
# print("After undersampling:", Counter(train_df['overall']))


In [5]:
def remove_punctuation(text):
    # Define the pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Replace punctuation with an empty string
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    # Normalize whitespace
    normalized_text = re.sub(r'\s+', ' ', text_without_punctuation)
    return normalized_text

# Text Processing
def preprocess_text(text, method='snowballstemming'):
    if not isinstance(text, str):
        return ''
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())

    # Define the set of stopwords
    stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he',
                'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its',
                'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
                'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
                'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
                'but', 'if','or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                'against', 'between', 'into', 'through','during', 'before', 'after', 'above', 'below', 'to', 'from',
                'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
                'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both','each', 'few', 'more', 'most', 'other',
                'some', 'such','product', 'purchase', 'buy', 'bought', 'ordered', 'order', 'item', 'items', 
                'review', 'reviews', 'star', 'stars', 'rating', 'ratings', 'customer', 'customers', 
                'seller', 'sellers', 'seller', 'brand', 'brands', 'price', 'prices', 'money', 
                'value', 'quality', 'deal', 'deals','shipping', 'shipping', 'delivery', 'delivered', 'package', 
                'packaging','customer service', 'service', 'support', 'warranty', 'warranties', 
                'problem','use','one','work','machine','using','need','needed','cut','recieve',
                'glitter','pen','marker','folder','card', 'look','looking','color','background',
                'tool','design','made','craft','piece','store','size','markers','page','make','used' }


    tokens = [remove_punctuation(token) for token in tokens if token.isalnum() and token not in stop_words]
    
    if method == 'snowballstemming':
        stemmer = nltk.stem.SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'porterstemming':
        stemmer = nltk.stem.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'lemmatization':
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [6]:
train_df.dropna(subset='Review',inplace=True)
train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')
# train_df['psprocessing_text'] = train_df['Review'].apply(preprocess_text, method='porterstemming')
# train_df['lemprocessing_text'] = train_df['Review'].apply(preprocess_text, method='lemmatization')

C:\Users\ishaq\AppData\Local\Temp\ipykernel_10040\3045761950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='Review',inplace=True)
C:\Users\ishaq\AppData\Local\Temp\ipykernel_10040\3045761950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')


In [7]:
train_df.dropna(subset='Review',inplace=True)
train_df.dropna(subset='ssprocessing_text',inplace=True)
# train_df.dropna(subset='psprocessing_text',inplace=True)
# train_df.dropna(subset='lemprocessing_text',inplace=True)
train_df = train_df[train_df['Review'].str.len() >= 3]
train_df = train_df[train_df['ssprocessing_text'].str.len() >= 3]
# train_df = train_df[train_df['psprocessing_text'].str.len() >= 3]
# train_df = train_df[train_df['lemprocessing_text'].str.len() >= 3]
train_df

C:\Users\ishaq\AppData\Local\Temp\ipykernel_10040\1420501431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='Review',inplace=True)
C:\Users\ishaq\AppData\Local\Temp\ipykernel_10040\1420501431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='ssprocessing_text',inplace=True)


,overall,Review,ssprocessing_text
0,5,I love these glitter pens. They sparkle deligh...,love pen sparkl delight brilliant colour even ...
1,5,It works well with my machine. I use mostly c...,work well most cone
2,5,"This is a great assortment of colors, though t...",great assort color though there lot pink mix s...
3,5,Just what I was looking for.,just
4,5,I make 400 birds for the hospital each month.,400 bird hospit each month
...,...,...,...
370858,5,I love these dies they make great backgrounds ...,love die great background card
370859,5,I love Darice embossing folders. Darcie's fol...,love daric emboss folder darci folder veri rea...
370860,5,I ordered these to add to my Earthy Markers th...,add earthi previous want brighter primari colo...
370861,4,Made perfect off white color by blending a tin...,perfect white blend tini amount yellow 2 white...


# Model Testing


In [8]:
vector_size = 50
window = 15
min_count = 10

train_subset = train_df[:]

train_data, test_data = train_test_split(train_subset, test_size=0.01, random_state=42)

sentences = [text.split() for text in train_data['ssprocessing_text']]
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['ssprocessing_text'])

X_train_tfidf = count_vectorizer.transform(train_data['ssprocessing_text'])
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text'])

In [9]:

def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [10]:
X_train_embeddings = generate_embeddings_batch(train_data['ssprocessing_text'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['ssprocessing_text'], model_fasttext)

X_train_count_sparse = X_train_tfidf.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [11]:
classifier = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=6000,verbose=1,n_jobs=-1)
classifier.fit(X_train_combined, train_data['overall'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


In [ ]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['Logistic Regression (FastText + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

In [ ]:
report = classification_report(test_data['overall'], y_pred)
print(report)

In [ ]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])

In [ ]:
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test
test_df.drop(columns=['Review','ssprocessing_text'],inplace=True)
test_df.to_csv('snowball_preprocessed_balanced_predictions.csv', index=False)